In [1]:
import torch
import os
import random
import os
import numpy as np
import logging
from config import Config
from model import TorchModel, choose_optimizer
from evaluate import Evaluator
from loader import load_data2
#[DEBUG, INFO, WARNING, ERROR, CRITICAL]
logging.basicConfig(level=logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
seed = Config["seed"] 
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
"""
模型训练主程序
"""
config = Config
#创建保存模型的目录
if not os.path.isdir(config["model_path"]):
    os.mkdir(config["model_path"])
#加载训练数据
train_data = load_data2(config["train_data_path"], config)
#加载模型
model = TorchModel(config)
# 标识是否使用gpu
cuda_flag = torch.cuda.is_available()
if cuda_flag:
    logger.info("gpu可以使用，迁移模型至gpu")
    model = model.cuda()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-28 19:31:14,410 - __main__ - INFO - gpu可以使用，迁移模型至gpu


In [ ]:
#加载优化器
optimizer = choose_optimizer(config, model)
#加载效果测试类
evaluator = Evaluator(config, model, logger)
#训练
for epoch in range(config["epoch"]):
    epoch += 1
    model.train()
    logger.info("epoch %d begin" % epoch)
    train_loss = []
    for index, batch_data in enumerate(train_data):
        if cuda_flag:
            batch_data = [d.cuda() for d in batch_data]

        optimizer.zero_grad()
        input_ids, labels = batch_data   #输入变化时这里需要修改，比如多输入，多输出的情况
        loss = model(input_ids, labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        if index % int(len(train_data) / 2) == 0:
            logger.info("batch loss %f" % loss)
    logger.info("epoch average loss: %f" % np.mean(train_loss))
    acc = evaluator.eval(epoch)

In [2]:
import torch
import os
import random
import os
import numpy as np
import logging
from config import Config
from model import TorchModel, choose_optimizer
from evaluate import Evaluator
from loader import load_data2
#[DEBUG, INFO, WARNING, ERROR, CRITICAL]
logging.basicConfig(level=logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
seed = Config["seed"] 
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
def main(config):
    """
    模型训练主程序
    """
    config = Config
    #创建保存模型的目录
    if not os.path.isdir(config["model_path"]):
        os.mkdir(config["model_path"])
    #加载训练数据
    train_data = load_data2(config["train_data_path"], config)
    #加载模型
    model = TorchModel(config)
    # 标识是否使用gpu
    cuda_flag = torch.cuda.is_available()
    if cuda_flag:
        logger.info("gpu可以使用，迁移模型至gpu")
        model = model.cuda()
        #加载优化器
    optimizer = choose_optimizer(config, model)
    #加载效果测试类
    evaluator = Evaluator(config, model, logger)
    #训练
    acc_max = []
    for epoch in range(config["epoch"]):
        epoch += 1
        model.train()
        logger.info("epoch %d begin" % epoch)
        train_loss = []
        for index, batch_data in enumerate(train_data):
            if cuda_flag:
                batch_data = [d.cuda() for d in batch_data]

            optimizer.zero_grad()
            input_ids, labels = batch_data   #输入变化时这里需要修改，比如多输入，多输出的情况
            loss = model(input_ids, labels)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            if index % int(len(train_data) / 2) == 0:
                logger.info("batch loss %f" % loss)
        logger.info("epoch average loss: %f" % np.mean(train_loss))
        acc = evaluator.eval(epoch)
        acc_max.append(acc_max)
    print('最大值:', np.max(acc_max))
    return None

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Config["learning_rate"] = 1e-5
Config["batch_size"] = 32
Config["pooling_style"] = "max"
main(Config)

2024-07-28 19:47:42,026 - __main__ - INFO - gpu可以使用，迁移模型至gpu
2024-07-28 19:47:45,668 - __main__ - INFO - epoch 1 begin
2024-07-28 19:47:47,634 - __main__ - INFO - batch loss 0.717829
2024-07-28 19:48:07,102 - __main__ - INFO - batch loss 0.274572
2024-07-28 19:48:26,681 - __main__ - INFO - epoch average loss: 0.342351
2024-07-28 19:48:26,682 - __main__ - INFO - 开始测试第1轮模型效果：
2024-07-28 19:48:38,361 - __main__ - INFO - 预测集合条目总量：5587
2024-07-28 19:48:38,361 - __main__ - INFO - 预测正确条目：4883，预测错误条目：704
2024-07-28 19:48:38,362 - __main__ - INFO - 预测准确率：0.873993
2024-07-28 19:48:38,362 - __main__ - INFO - --------------------
2024-07-28 19:48:38,364 - __main__ - INFO - epoch 2 begin
2024-07-28 19:48:38,566 - __main__ - INFO - batch loss 0.182868
2024-07-28 19:48:58,909 - __main__ - INFO - batch loss 0.289530
2024-07-28 19:49:19,082 - __main__ - INFO - epoch average loss: 0.235641
2024-07-28 19:49:19,082 - __main__ - INFO - 开始测试第2轮模型效果：
2024-07-28 19:49:30,869 - __main__ - INFO - 预测集合条目总量：5587


In [ ]:
Config["learning_rate"] = 1e-3
Config["batch_size"] = 32
Config["pooling_style"] = "max"
main(Config)

2024-07-28 19:53:25,641 - __main__ - INFO - gpu可以使用，迁移模型至gpu
2024-07-28 19:53:28,187 - __main__ - INFO - epoch 1 begin
2024-07-28 19:53:29,273 - __main__ - INFO - batch loss 0.717829
2024-07-28 19:53:48,999 - __main__ - INFO - batch loss 0.693204
2024-07-28 19:54:08,872 - __main__ - INFO - epoch average loss: 0.723530
2024-07-28 19:54:08,873 - __main__ - INFO - 开始测试第1轮模型效果：
2024-07-28 19:54:20,416 - __main__ - INFO - 预测集合条目总量：5587
2024-07-28 19:54:20,417 - __main__ - INFO - 预测正确条目：811，预测错误条目：4776
2024-07-28 19:54:20,418 - __main__ - INFO - 预测准确率：0.145158
2024-07-28 19:54:20,418 - __main__ - INFO - --------------------
2024-07-28 19:54:20,419 - __main__ - INFO - epoch 2 begin
2024-07-28 19:54:20,624 - __main__ - INFO - batch loss 0.677666
2024-07-28 19:54:40,921 - __main__ - INFO - batch loss 0.730521
2024-07-28 19:55:00,845 - __main__ - INFO - epoch average loss: 0.701788
2024-07-28 19:55:00,845 - __main__ - INFO - 开始测试第2轮模型效果：
2024-07-28 19:55:12,190 - __main__ - INFO - 预测集合条目总量：5587


In [ ]:
Config["learning_rate"] = 1e-5
Config["batch_size"] = 128
Config["pooling_style"] = "max"
main(Config)

2024-07-28 19:58:52,405 - __main__ - INFO - gpu可以使用，迁移模型至gpu
2024-07-28 19:58:54,949 - __main__ - INFO - epoch 1 begin
2024-07-28 19:58:56,447 - __main__ - INFO - batch loss 0.711806
2024-07-28 19:59:13,318 - __main__ - INFO - batch loss 0.318248
2024-07-28 19:59:29,789 - __main__ - INFO - epoch average loss: 0.428631
2024-07-28 19:59:29,789 - __main__ - INFO - 开始测试第1轮模型效果：
2024-07-28 19:59:40,846 - __main__ - INFO - 预测集合条目总量：5587
2024-07-28 19:59:40,847 - __main__ - INFO - 预测正确条目：5127，预测错误条目：460
2024-07-28 19:59:40,847 - __main__ - INFO - 预测准确率：0.917666
2024-07-28 19:59:40,847 - __main__ - INFO - --------------------
2024-07-28 19:59:40,849 - __main__ - INFO - epoch 2 begin
2024-07-28 19:59:41,546 - __main__ - INFO - batch loss 0.229748
2024-07-28 19:59:59,093 - __main__ - INFO - batch loss 0.313779
2024-07-28 20:00:15,893 - __main__ - INFO - epoch average loss: 0.272037
2024-07-28 20:00:15,894 - __main__ - INFO - 开始测试第2轮模型效果：
2024-07-28 20:00:26,948 - __main__ - INFO - 预测集合条目总量：5587


In [ ]:
Config["learning_rate"] = 1e-5
Config["batch_size"] = 128
Config["pooling_style"] = "avg"
main(Config)

2024-07-28 20:03:16,718 - __main__ - INFO - gpu可以使用，迁移模型至gpu
2024-07-28 20:03:19,297 - __main__ - INFO - epoch 1 begin
2024-07-28 20:03:20,928 - __main__ - INFO - batch loss 0.759782
2024-07-28 20:03:37,963 - __main__ - INFO - batch loss 0.300041
2024-07-28 20:03:54,625 - __main__ - INFO - epoch average loss: 0.410912
2024-07-28 20:03:54,626 - __main__ - INFO - 开始测试第1轮模型效果：
2024-07-28 20:04:05,840 - __main__ - INFO - 预测集合条目总量：5587
2024-07-28 20:04:05,840 - __main__ - INFO - 预测正确条目：4968，预测错误条目：619
2024-07-28 20:04:05,841 - __main__ - INFO - 预测准确率：0.889207
2024-07-28 20:04:05,841 - __main__ - INFO - --------------------
2024-07-28 20:04:05,843 - __main__ - INFO - epoch 2 begin
2024-07-28 20:04:06,545 - __main__ - INFO - batch loss 0.205989
2024-07-28 20:04:24,033 - __main__ - INFO - batch loss 0.307683
2024-07-28 20:04:40,703 - __main__ - INFO - epoch average loss: 0.266886
2024-07-28 20:04:40,704 - __main__ - INFO - 开始测试第2轮模型效果：
2024-07-28 20:04:51,687 - __main__ - INFO - 预测集合条目总量：5587
